In [1]:
# Create dragon plot and other stuff
# See .chat:[PRocess-Induced Surrogate Modeling/PRISM]
import gpjax as gpx
import jax
import jax.numpy as jnp
import numpy as np
import optax as ox
import plotly.graph_objects as go
from flax import nnx
from gpjax.dataset import Dataset
from gpjax.fit import get_batch
from gpjax.parameters import Parameter
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

from prism.pack import NormalizedPACK
from prism.svi import (
    batch_collapsed_elbo_masked,
    gp_posterior_mean_from_eps,
    pick_best,
)
from surrogate.prism import get_train_data
from utils import time_this
from utils.jax import vk

master_key = jax.random.PRNGKey(0)


In [2]:
num_train_samples = 5000

X, y, oq = get_train_data(n=num_train_samples)
N, WIDTH = X.shape  # Number of waveforms in dataset, max waveform length

dataset = Dataset(X=X, y=y)


[Memory]0.6s, 0.0min    : Loading get_train_data...


In [3]:
test_batch = get_batch(dataset, 3, vk())
lengths = jnp.sum(~jnp.isnan(test_batch.y), axis=1)
fig = go.Figure()
for i in range(test_batch.X.shape[0]):
    fig.add_trace(
        go.Scatter(
            x=np.asarray(test_batch.X[i]),
            y=np.asarray(test_batch.y[i]),
            mode="lines+markers",
            name=f"sample {i}",
        )
    )
fig.update_layout(
    title=f"Random batch with lengths = {lengths}",
    xaxis_title="tau",
    yaxis_title="du",
)
fig.show()


In [4]:
kernel = NormalizedPACK(d=1)
NUM_INDUCING = 16

meanf = gpx.mean_functions.Zero()
likelihood = gpx.likelihoods.Gaussian(num_datapoints=WIDTH)
prior = gpx.gps.Prior(mean_function=meanf, kernel=kernel)
posterior = prior * likelihood


def get_variational_model(z=None, num_inducing_svi=NUM_INDUCING):
    if z is None:
        z = jnp.zeros((num_inducing_svi, 1))

    q = gpx.variational_families.CollapsedVariationalGaussian(
        posterior=posterior, inducing_inputs=z
    )
    return q


batch_size = 256
num_iters = 5000
num_restarts = 1

lr = 1e-3

master_key, subkey = jax.random.split(master_key)
keys = jax.random.split(subkey, num_restarts)


def optimize(key, num_inducing_svi=NUM_INDUCING):
    key, subkey = jax.random.split(key)

    z = jax.random.uniform(
        subkey, shape=(num_inducing_svi, 1), minval=0.0, maxval=1.0
    )

    q = get_variational_model(z)

    opt_posterior, history = gpx.fit(
        model=q,
        objective=lambda p, d: -batch_collapsed_elbo_masked(p, d, N),
        train_data=dataset,
        optim=ox.adam(learning_rate=lr),
        num_iters=num_iters,
        key=key,
        batch_size=batch_size,
        trainable=Parameter,
    )

    return nnx.state(opt_posterior), history


with time_this() as timer:
    states, histories = jax.vmap(optimize)(keys)

fig = go.Figure()
for i, run in enumerate(np.asarray(-histories)):
    fig.add_trace(go.Scatter(y=run, mode="lines", name=f"run {i}"))
fig.update_layout(
    title="ELBO runs during training",
    xaxis_title="iteration",
    yaxis_title="ELBO",
)
fig.show()


  0%|          | 0/5000 [00:00<?, ?it/s]

Walltime: 196.078 s


In [5]:
# pick best run
opt_posterior, history = pick_best(states, histories, get_variational_model())

fig = go.Figure()
fig.add_trace(go.Scatter(y=np.asarray(-history), mode="lines", name="best run"))
fig.update_layout(
    title="ELBO during training (best run)",
    xaxis_title="iteration",
    yaxis_title="ELBO",
)
fig.show()


In [6]:
print(
    "Observation sigma_noise:",
    opt_posterior.posterior.likelihood.obs_stddev[...],
)


Observation sigma_noise: 0.18085671673666207


In [7]:
from utils.jax import safe_cholesky

Z = opt_posterior.inducing_inputs
Kmm = opt_posterior.posterior.prior.kernel.gram(Z).to_dense()
Lzz = safe_cholesky(Kmm)  # Kmm = Lzz @ Lzz.T


def psi(t):
    x = jnp.asarray(t).reshape(1, -1)
    Kxz = kernel.cross_covariance(x, Z)
    return jax.scipy.linalg.solve_triangular(
        Lzz, Kxz.T, lower=True
    ).squeeze()  # psi = Lzz^{-1} Kzx


t = jnp.linspace(0, 1, 500)
Psi = jax.vmap(psi)(t)

eps = jax.random.normal(vk(), shape=(NUM_INDUCING, 5))
y = Psi @ eps
fig = go.Figure()
for i in range(y.shape[1]):
    fig.add_trace(
        go.Scatter(x=np.asarray(t), y=np.asarray(y[:, i]), mode="lines")
    )
fig.update_layout(
    title="Samples of learned latent function distribution",
    xaxis_title="tau",
)
fig.show()
# This is a prior draw from the learned RKHS subspace, not data-like yet.
# It answers: What does a typical GP draw look like under the learned kernel?
# expected to look generic and smooth


In [8]:
fig = go.Figure()
for i in range(Psi.shape[1]):
    fig.add_trace(
        go.Scatter(
            x=np.asarray(t),
            y=np.asarray(Psi[:, i]),
            mode="lines",
            showlegend=False,
        )
    )
fig.update_layout(
    title="Learned basis functions psi_m(t)",
    xaxis_title="tau",
)
fig.show()


In [9]:
# Now we test if the learned RKHS is rich enough to reconstruct some test waveforms
from prism.svi import infer_eps_posterior_single

test_index = 321

mu_eps, Sigma_eps = infer_eps_posterior_single(
    opt_posterior, dataset.X[test_index], dataset.y[test_index]
)


f_mean = gp_posterior_mean_from_eps(
    opt_posterior,
    t,
    mu_eps,
)


fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.asarray(dataset.X[test_index]),
        y=np.asarray(dataset.y[test_index]),
        mode="lines",
        name="Data",
    )
)
fig.add_trace(
    go.Scatter(
        x=np.asarray(t),
        y=np.asarray(f_mean),
        mode="lines",
        name="Posterior mean",
    )
)
fig.show()


In [ ]:
# Save GPU memory
with jax.default_device(jax.devices("cpu")[0]):
    mu_eps, Sigma_eps = jax.vmap(
        infer_eps_posterior_single,
        in_axes=(None, 0, 0),
    )(opt_posterior, dataset.X, dataset.y)


In [11]:
X_latent = StandardScaler().fit_transform(mu_eps)

X_2d = TSNE(
    n_components=2,
    perplexity=30,
    learning_rate="auto",
    init="pca",
    random_state=0,
).fit_transform(X_latent)

# scatter and color by Oq
fig = go.Figure(
    go.Scatter(
        x=np.asarray(X_2d[:, 0]),
        y=np.asarray(X_2d[:, 1]),
        mode="markers",
        marker=dict(
            color=np.asarray(oq),
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="OQ (open quotient)"),
        ),
    )
)
fig.update_layout(
    title="t-SNE of inferred latent amplitudes colored by OQ",
    xaxis_title="t-SNE dim 1",
    yaxis_title="t-SNE dim 2",
)
fig.show()


In [12]:
# Uncertaintly confirms non-degenerate uncertainty, so MoPPCA with uncertainty makes sense
fig = go.Figure(go.Histogram(x=np.asarray(jnp.trace(Sigma_eps.T))))
fig.update_layout(title="Histogram of trace of posterior covariance Sigma_eps")
fig.show()


In [13]:
# Let us first get a scree plot of data resampled to same grid to get an idea of underlying linear dimension
# resample all samples to a common tau grid
from tqdm import tqdm

from surrogate import source
from surrogate.prism import extract_train_data

lf_samples = source.get_lf_samples(10_000)[:num_train_samples]

N_tau = WIDTH
tau_grid = np.linspace(0.0, 1.0, N_tau)

du_tau = []
for s_tau, s_du, _ in tqdm(list(extract_train_data(lf_samples))):
    du_tau.append(np.interp(tau_grid, s_tau, s_du))


[Memory]322.1s, 5.4min  : Loading get_lf_samples...


100%|██████████| 5000/5000 [00:07<00:00, 643.37it/s] 


In [14]:
# do PCA on resampled data
# We can see manifold gradient as expected: OQ varies smoothly
from sklearn.decomposition import PCA

du_tau = np.stack(du_tau)
pca = PCA().fit(du_tau)

# now do cumulative scree plot
explained_variance_ratio_cumsum = jnp.cumsum(pca.explained_variance_ratio_)
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.asarray(jnp.arange(1, len(explained_variance_ratio_cumsum) + 1)),
        y=np.asarray(explained_variance_ratio_cumsum),
        mode="lines",
        name="explained variance",
    )
)
fig.add_hline(y=0.95, line_color="red", line_dash="dash")
fig.update_layout(
    title="Scree plot of LF waveform data",
    xaxis_title="Number of principal components",
    yaxis_title="Cumulative explained variance ratio",
)
fig.update_xaxes(type="log")
fig.show()
